In [1]:
import sys
from pathlib import Path
import json

parent_dir = Path("/home/ujx4ab/ondemand/CBM_Final_Project/src/LLM")
sys.path.append(str(parent_dir))

from data_utils import load_reddit_posts, get_comments_for_post, get_user_comment_history, get_comments_in_post_thread, upload_json_to_file
from Mistral.minstral_utils import zero_shot_json_to_skeleton, run_prompt_on_mistral

/home/ujx4ab/ondemand/CBM_Final_Project/.venv/lib64/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [17]:
from minstral_utils import load_quantized_model, initialize_tokenizer
model_name = 'mistralai/Mistral-7B-Instruct-v0.3'
model = load_quantized_model(model_name)
tokenizer = initialize_tokenizer(model_name)

def run_prompt_on_mistral(prompt):
    stop_token_ids = [0]
    encoded = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
    model_input = encoded
    generated_ids = model.generate(**model_input, max_new_tokens=200, do_sample=True)
    decoded = tokenizer.batch_decode(generated_ids)

    return decoded[0].split("[/INST]", 1)[1].strip()
        

In [3]:
# Data filepaths
post_fp = '/home/ujx4ab/ondemand/CBM_Final_Project/Data/test_data/r_books_posts.csv'
post_interaction_fp = '/home/ujx4ab/ondemand/CBM_Final_Project/Data/test_data/r_books_comments_cleaned_and_pruned.csv'
user_comment_history_fp = '/home/ujx4ab/ondemand/CBM_Final_Project/Data/test_data/r_books_comments_cleaned.csv'

In [18]:
# get posts
posts = load_reddit_posts(post_fp)

# loop through posts to generate prompts ... rn limiting it to 1
for i in range(1): 
    post = posts[i]
    post_title = post['title']
    post_content = post['content']

    # get post graph
    post_graph = get_comments_for_post(post_interaction_fp, post['post_id'])

    # for each comment in the post graph
    for comment in post_graph:
        # get the comment ID
        comment_id = comment['comment_id']
        print("comment ID ", comment_id)

        # get the users comment history
        user_history = get_user_comment_history(user_comment_history_fp, comment['user'], timestamp=comment['timestamp'], post_id=post['post_id'])
        user_comment_history = ""
        for comment in user_history:
            user_comment_history += comment['comment_text'] + "\n"

        # get the thread of comments preceeding the current comment of interest
        thread = get_comments_in_post_thread(post_interaction_fp, 'kfth4pc')
        previous_comments_on_thread = []
        for row in thread: 
            previous_comments_on_thread.append(row['comment_text'])
        previous_comments_on_thread.reverse()

        comment_of_interest = previous_comments_on_thread[-1]
        previous_comments_on_thread = previous_comments_on_thread[:-1]

        previous_comments_on_thread = "\n".join(previous_comments_on_thread)
        #print(previous_comments_on_thread)

        upload_json_to_file(
            post_title=post_title, 
            post_content=post_content, 
            previous_comments_on_thread=previous_comments_on_thread, 
            comment_of_interest = comment_of_interest,
            user_comment_history=user_comment_history, 
            comment_id=comment_id
        )
        promt_json_fp = f"/home/ujx4ab/ondemand/CBM_Final_Project/Data/post_prompts/{post_title}.json"

        # Load the JSON file
        with open(promt_json_fp, 'r') as file:
            data = json.load(file)

        prompt = zero_shot_json_to_skeleton(data)
        response = run_prompt_on_mistral(prompt)
        
        print(response)


comment ID  kftg5ci


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
/home/ujx4ab/ondemand/CBM_Final_Project/.venv/lib64/python3.11/site-packages/transformers/generation/utils.py:2097: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


The user seems to be sharing a list of their least favorite books they read in 2023. Some of the titles do not correspond to actual books published in that year, suggesting that the list may be an amalgamation of books the user read during the year, not necessarily books published in that year. The user provides summaries of the books, indicating dissatisfaction with the plot, characters, and overall writing style.

     In response to the comment of interest, the user acknowledges that they did indeed read the books on their list, despite not enjoying them. However, they clarify that they did not necessarily finish reading each book due to their dissatisfaction with them. The user mentions that they usually lean towards reading mysteries and thrillers.

     It is uncertain how the user's comment history relates to their book reviews on this particular thread, as the comment history does not seem directly related to the topic of
comment ID  kfth4pc


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Given the information above, here is an English response to the comment of interest: The user has read all the books they mentioned, but the majority of them were not enjoyable and were DNF'd (Did Not Finish) due to lack of interest. These books had poor plot development, lack of suspense, over-use of cliches, and uninteresting characters. The user's comment history shows that they have strict standards for their reading material and will not waste time on books that do not meet their expectations. The user's preferences lean towards mysteries and thrillers rather than romance or fantasy novels.</s>
comment ID  kfth6ym


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Given the user's comment on their worst reads of 2023 and their stated preference for mystery and thriller genres, it's likely that the user has read some of the books listed in the post they shared. However, it's important to note that the user mentions that they don't finish reading books they absolutely hate, so it's possible that they only read part of some of the books on the list.

     It's also worth mentioning that the user commented on their girlfriend's reading preferences for books by Sarah J. Maas, which are not included on the list of worst reads of 2023. It's possible that the user has read some of these books but did not find them to be their worst reads of the year.

     In terms of the comment of interest, the user specifically mentioned that they have read the books listed in the post, even if they didn't finish them
comment ID  kfti0kt


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


KeyboardInterrupt: 